### Download Dataset

In [18]:
# %%capture
# !wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EbMhBPnmIb5MutZvGicPKggBWKm5hLs0iwKfGW7_TwQIKg?download=1 -O custom_korean_family_dataset_resolution_128.zip
# !unzip custom_korean_family_dataset_resolution_128.zip -d ./custom_korean_family_dataset_resolution_128

In [1]:
import os
import time
import random
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from trainer import train, test
from evaluation.mia import MIA
from evaluation.accuracy import calculate_accuracy, calculate_all_accuracies
from model import load_model
from MUFAC_loader import get_data_loaders
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define paths
train_meta_data_path = "/home/dasol/Unlearning/custom_korean_family_dataset_resolution_128_2/custom_train_dataset.csv"
train_image_directory = "/home/dasol/Unlearning/custom_korean_family_dataset_resolution_128_2/train_images"

val_meta_data_path = "/home/dasol/Unlearning/custom_korean_family_dataset_resolution_128_2/custom_val_dataset.csv"
val_image_directory = "/home/dasol/Unlearning/custom_korean_family_dataset_resolution_128_2/val_images"

test_meta_data_path = "/home/dasol/Unlearning/custom_korean_family_dataset_resolution_128_2/custom_test_dataset.csv"
test_image_directory = "/home/dasol/Unlearning/custom_korean_family_dataset_resolution_128_2/test_images"

batch_size = 32
data_loaders = get_data_loaders(
    train_meta_data_path, train_image_directory,
    val_meta_data_path, val_image_directory,
    test_meta_data_path, test_image_directory, batch_size
)

print(f"Number of training samples: {len(data_loaders['train'].dataset)}")
print(f"Number of test samples: {len(data_loaders['test'] .dataset)}")
print(f"Number of val samples: {len(data_loaders['val'].dataset)}")
print(f"Number of retained train samples: {len(data_loaders['retain_train'].dataset)}")
print(f"Number of forgotten train samples: {len(data_loaders['forget_train'].dataset)}")

Number of training samples: 10025
Number of test samples: 1504
Number of val samples: 1539
Number of retained train samples: 8525
Number of forgotten train samples: 1500


## Base Model

### Training Original Model

In [11]:
learning_rate = 0.01
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = load_model(8, device, model_path=None)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=200)

num_epochs = 100
best_val_acc = 0
best_epoch = 0

history = []
accuracy = []
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, data_loaders['train'], criterion, optimizer, scheduler, device, epoch)
    val_loss, val_acc = test(model, data_loaders['val'], criterion, device)
    history.append((train_loss, train_acc))
    accuracy.append((val_loss, val_acc))

    if val_acc > best_val_acc:
        print("[Info] best test accuracy!")
        best_val_acc = val_acc
        best_epoch = epoch
        #torch.save(model.state_dict(), f'checkpoints/resnet/best_{epoch + 1}_age_original.pth')

#torch.save(model.state_dict(), f'checkpoints/resnet/last_{num_epochs}_age_original.pth')

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_path =  'checkpoint/last_100_age_original.pth'
model = load_model(8, device, model_path)
criterion = nn.CrossEntropyLoss()

test_acc = calculate_accuracy(model, data_loaders['test'], criterion, device)
mia = MIA(model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

Test Acc: {'Loss': 0.0374256043516575, 'Acc': 0.6329787234042553, 'Top-2 Acc': 0.8803191489361702}
MIA: {'MIA Regression Accuracy': 0.7618180241131061, 'MIA CV Accuracy': 0.6923588039867109, 'Forgetting Score': 0.1923588039867109}
Final score: 0.6241305577154167


### Training Retrained Model

In [24]:
learning_rate = 0.01
retrained_model = load_model(8, device, model_path=None)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(retrained_model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=200)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_epochs = 100
best_val_acc = 0
best_epoch = 0

history = []
accuracy = []
for epoch in range(num_epochs):
    train_loss, train_acc = train(retrained_model, data_loaders['retain_train'], criterion, optimizer, scheduler, device, epoch)
    val_loss, val_acc = test(retrained_model, data_loaders['val'], criterion, device)
    history.append((train_loss, train_acc))
    accuracy.append((val_loss, val_acc))

    if val_acc > best_val_acc:
        print("[Info] best test accuracy!")
        best_val_acc = val_acc
        best_epoch = epoch
        #torch.save(retrained_model.state_dict(), f'checkpoint/best_{epoch + 1}_age_retrained.pth')

#torch.save(retrained_model.state_dict(), f'checkpoint/last_{num_epochs}_age_retrained.pth')

In [23]:
model_path = 'checkpoint/last_100_age_retrained.pth'
retrained_model = load_model(8, device, model_path)
criterion = nn.CrossEntropyLoss()

test_acc = calculate_accuracy(retrained_model, data_loaders['test'], criterion, device)
mia = MIA(retrained_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

Test Acc: {'Loss': 0.037642640834476084, 'Acc': 0.6083776595744681, 'Top-2 Acc': 0.8656914893617021}
MIA: {'MIA Regression Accuracy': 0.7678896695290138, 'MIA CV Accuracy': 0.5548172757475083, 'Forgetting Score': 0.054817275747508276}
Final score: 0.7493715540397258


## Unlearning Experiments

### Fine-tunning

In [3]:
model_path =  'checkpoint/last_100_age_original.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
finetune_model = load_model(8, device, model_path)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(finetune_model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
num_epochs=2
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
for epoch in range(num_epochs):
    train_loss, train_acc = train(finetune_model, data_loaders['retain_train'], criterion, optimizer, scheduler, device, epoch)
    val_loss, val_acc = test(finetune_model, data_loaders['val'], criterion, device)

test_acc = calculate_accuracy(finetune_model, data_loaders['test'], criterion, device)
mia = MIA(finetune_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

[Epoch: 1 - Training]
[Batch: 80] running train loss: 0.010473815829027445, running train accuracy: 0.888671875
[Batch: 160] running train loss: 0.00998851096665021, running train accuracy: 0.8935546875
[Batch: 240] running train loss: 0.009844225030974485, running train accuracy: 0.89348965883255
train loss: 0.009800257664446957, accuracy: 0.8932551145553589
elapsed time: 15.528449296951294
[Test]
[Batch: 1] running test loss: 0.0499936044216156, running test accuracy: 0.59375
test loss: 0.045445403005489673, accuracy: 0.5724496245384216
elapsed time: 1.0893614292144775
[Epoch: 2 - Training]
[Batch: 80] running train loss: 0.008728485857136547, running train accuracy: 0.903124988079071
[Batch: 160] running train loss: 0.008625717423274181, running train accuracy: 0.8994140625
[Batch: 240] running train loss: 0.008521037284905712, running train accuracy: 0.9027343988418579
train loss: 0.008530318191673748, accuracy: 0.9031084775924683
elapsed time: 15.045075178146362
[Test]
[Batch: 1] 

### CF-K

In [3]:
from unlearn.cfk import cfk_train

model_path =  'checkpoint/last_100_age_original.pth'
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
criterion = nn.CrossEntropyLoss()
epochs=2
cfk_model = load_model(8, device, model_path)
cfk_model = cfk_train(cfk_model, data_loaders['retain_train'], data_loaders['test'], device, epochs)

test_acc = calculate_accuracy(cfk_model, data_loaders['test'], criterion, device)
mia = MIA(cfk_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

[Epoch: 1 - Training]
[Batch: 80] running train loss: 0.011000621796119959, running train accuracy: 0.876171886920929
[Batch: 160] running train loss: 0.010439964865508956, running train accuracy: 0.885937511920929
[Batch: 240] running train loss: 0.010353822650116248, running train accuracy: 0.8878906965255737
train loss: 0.010294202425437939, accuracy: 0.8877419233322144
elapsed time: 11.836766481399536
[Test]
[Batch: 1] running test loss: 0.03943987563252449, running test accuracy: 0.59375
test loss: 0.037975033626277396, accuracy: 0.6309840083122253
elapsed time: 0.9689269065856934
Epoch [1/2] - Train Loss: 0.0103, Train Accuracy: 0.89%
Epoch [1/2] - Test Loss: 0.0380, Test Accuracy: 0.63%
[Epoch: 2 - Training]
[Batch: 80] running train loss: 0.009430807890021242, running train accuracy: 0.8960937857627869
[Batch: 160] running train loss: 0.010034417739370837, running train accuracy: 0.891406238079071
[Batch: 240] running train loss: 0.01028650209773332, running train accuracy: 0.8

### EU-K

In [7]:
from unlearn.euk import euk_unlearn

epochs = 2
model_path =  'checkpoint/last_100_age_original.pth'
euk_model = load_model(8, device, model_path)
euk_model = euk_unlearn(euk_model, data_loaders['retain_train'], device, epochs)

test_acc = calculate_accuracy(euk_model, data_loaders['test'], criterion, device)
mia = MIA(euk_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

Unlearning Epoch [1/2] - Loss: 0.3320, Accuracy: 88.47%
Unlearning Epoch [2/2] - Loss: 0.3204, Accuracy: 89.04%
Test Acc: {'Loss': 0.03736931507654013, 'Acc': 0.629654255319149, 'Top-2 Acc': 0.8776595744680851}
MIA: {'MIA Regression Accuracy': 0.7611241217798594, 'MIA CV Accuracy': 0.6893687707641196, 'Forgetting Score': 0.18936877076411962}
Final score: 0.6254583568954548


### NegGrad

In [8]:
from unlearn.neg_grad import neggrad_unlearn

class Args:
    def __init__(self):
        self.gpu = 0
        self.lr = 0.001
        self.momentum = 0.9
        self.weight_decay = 5e-4
        self.num_classes = 8
        self.epochs = 1
        self.device = torch.device(f"cuda:{self.gpu}" if torch.cuda.is_available() else "cpu")

args = Args()
criterion = nn.CrossEntropyLoss()
model_path =  'checkpoint/last_100_age_original.pth'
neggrad_model = load_model(8, args.device, model_path)
neggrad_unlearn(neggrad_model, data_loaders['forget_train'], criterion, args)

test_acc = calculate_accuracy(neggrad_model, data_loaders['test'], criterion, args.device)
mia = MIA(neggrad_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

NegGrad Epoch [1/1] Loss: 0.846
Test Acc: {'Loss': 0.11845020474271571, 'Acc': 0.4115691489361702, 'Top-2 Acc': 0.7054521276595744}
MIA: {'MIA Regression Accuracy': 0.7394396738659034, 'MIA CV Accuracy': 0.4750830564784054, 'Forgetting Score': 0.024916943521594626}
Final score: 0.6808676309464905


### UNSIR

In [9]:
from unlearn.unsir import Noise, generate_error_maximizing_noise, impair_step, repair_step
from torch.utils.data import DataLoader

class Args:
    def __init__(self):
        self.alpha = 0.1
        self.num_classes = 8
        self.device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
        self.impair_epochs = 1
        self.repair_epochs = 2
        self.lr = 0.001

args = Args()
criterion = nn.CrossEntropyLoss()
model_path = 'checkpoint/last_100_age_original.pth'
unsir_model = load_model(8, device, model_path)

noise = generate_error_maximizing_noise(unsir_model, data_loaders['forget_train'], args.device, args.impair_epochs)

# Impair Step
noisy_data = [(noise()[i % noise().size(0)], data_loaders['forget_train'].dataset[i][1]) for i in range(len(data_loaders['forget_train'].dataset))]
noisy_loader = DataLoader(noisy_data, batch_size=128, shuffle=True)
impair_step(unsir_model, noisy_loader, args.device, args.lr, args.impair_epochs)

print("Performance of Impaired Model on Forget Samples")
test_acc = calculate_accuracy(unsir_model, data_loaders['test'], criterion, args.device)
mia_impaired = MIA(unsir_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
print(f'Test Acc: {test_acc}')
print(f'Impaired MIA : {mia_impaired}')

# Repair Step
repair_step(unsir_model, data_loaders['retain_train'], args.device, args.lr, args.repair_epochs)

print("Performance of Repaired Model on Forget Samples")
test_acc = calculate_accuracy(unsir_model, data_loaders['test'], criterion, args.device)
mia_repaired = MIA(unsir_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
final_score = (test_acc["Acc"] + 1 - abs(mia_repaired["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'Repaired MIA: {mia_repaired}')
print(f'Final score: {final_score}')

Epoch [1/1] - Noise Loss: 2923.9854
Impair Step - Train loss 1: 2.9501, Train Acc: 15.40%
Performance of Impaired Model on Forget Samples
Test Acc: {'Loss': 0.07558805432091369, 'Acc': 0.2646276595744681, 'Top-2 Acc': 0.3151595744680851}
Impaired MIA : {'MIA Regression Accuracy': 0.7394396738659034, 'MIA CV Accuracy': 0.5255813953488372, 'Forgetting Score': 0.025581395348837188}
Repair Step - Train loss 1: 1.2748, Train Acc: 47.91%
Repair Step - Train loss 2: 2.4218, Train Acc: 100.87%
Performance of Repaired Model on Forget Samples
Test Acc: {'Loss': 0.042461668478047596, 'Acc': 0.49069148936170215, 'Top-2 Acc': 0.726063829787234}
Repaired MIA: {'MIA Regression Accuracy': 0.739613149449215, 'MIA CV Accuracy': 0.5308970099667774, 'Forgetting Score': 0.030897009966777356}
Final score: 0.7144487347140738


### BadT

In [10]:
from unlearn.badT import badt

class Args:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = 42
        self.bt_optim = "adam"
        self.bt_alpha = 1
        self.bt_beta = 1
        self.bt_kd_T = 4
        self.bt_epochs = 2
        self.bt_learning_rate = 0.001
        self.bt_lr_decay_epochs = [10, 10, 10]
        self.bt_lr_decay_rate = 0.1
        self.bt_weight_decay = 5e-4
        self.bt_momentum = 0.9

args = Args()
criterion = nn.CrossEntropyLoss()
model_path = 'checkpoint/last_100_age_original.pth'
gteacher = load_model(8, args.device, model_path)
bteacher = load_model(8, args.device, model_path=None)
student = load_model(8, args.device, model_path)

badT_model = badt(
    gteacher=gteacher,
    bteacher=bteacher,
    student=student,
    retain_loader=data_loaders["retain_train"],
    forget_loader=data_loaders["forget_train"],
    valid_loader_full=data_loaders["test"],
    args=args
)

test_acc = calculate_accuracy(badT_model, data_loaders['test'], criterion, args.device)
mia = MIA(badT_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

==> Bad Teacher Unlearning ...
Epoch 1: train_acc: 35.79, train_loss: -675800.37
Epoch 2: train_acc: 30.71, train_loss: -27270869.84
Test Acc: {'Loss': 20565.733281237015, 'Acc': 0.42021276595744683, 'Top-2 Acc': 0.6941489361702128}
MIA: {'MIA Regression Accuracy': 0.7394396738659034, 'MIA CV Accuracy': 0.5627906976744186, 'Forgetting Score': 0.06279069767441858}
Final score: 0.6473156853043048


### SCRUB

In [12]:
import torch
from unlearn.scrub import scrub

class Args:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = 42
        self.optim = "sgd"  
        self.sgda_epochs = 2
        self.sgda_learning_rate = 0.0005
        self.sgda_weight_decay = 5e-4
        self.sgda_momentum = 0.9
        self.msteps = 2
        self.kd_T = 4  
        self.lr_decay_epochs = [3, 5, 9]
        self.lr_decay_rate = 0.1

args = Args()
model_path =  'checkpoint/last_100_age_original.pth'
teacher_model = load_model(8, args.device, model_path)
student_model = load_model(8, args.device, model_path)

scrub_model = scrub(
    teacher=teacher_model,
    student=student_model,
    args=args,
    retain_loader_train=data_loaders["retain_train"],
    retain_loader_test=data_loaders["retain_test"],
    forget_loader_train=data_loaders["forget_train"],
    forget_loader_test=data_loaders["forget_test"],
    valid_loader_full=data_loaders["test"] 
)

test_acc = calculate_accuracy(scrub_model, data_loaders['test'], criterion, args.device)
mia = MIA(scrub_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], args.device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA: {mia}')
print(f'Final score: {final_score}')

==> scrub unlearning ...
Epoch 1: maximize loss: -3.10, train_acc: 81.24, minimize loss: 1.67
==> scrub unlearning ...
Epoch 2: maximize loss: -2.71, train_acc: 82.97, minimize loss: 1.35
Total time: 55.80 seconds
Test Acc: {'Loss': 0.03583506833603407, 'Acc': 0.6462765957446809, 'Top-2 Acc': 0.8769946808510638}
MIA: {'MIA Regression Accuracy': 0.7619047619047619, 'MIA CV Accuracy': 0.6637873754152824, 'Forgetting Score': 0.16378737541528243}
Final score: 0.6593509224570581


### DLFD

In [36]:
from unlearn.dlfd import train_dlfd, set_seed, linear_weight_scheduler
set_seed(42) 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_path = 'checkpoint/last_100_age_original.pth'
dlfm_model = load_model(8, device, model_path)
path = 'checkpoint/age_dlfd.pth'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(dlfm_model.parameters(), lr=0.008, momentum=0.9, weight_decay=5e-4)

forget_weight = 3.0  # Weight for forgetting
initial_retain_weight = 0  # Initial weight for retaining
final_retain_weight = 1  # Final weight for retaining
perturbation_steps = 15  # Number of steps for perturbation
perturbation_strength = 0.4  # Strength of perturbation
forget_threshold = 0.15  # Forgetting score threshold
best_test_acc = 0  # Save the best test accuracy
max_forgetting_score = 0.09  # Forgetting score saving threshold

train_dlfd(
        dlfm_model, data_loaders, criterion, optimizer, device,
        perturbation_steps, perturbation_strength, forget_threshold, max_forgetting_score,
        initial_retain_weight, final_retain_weight, forget_weight, path
    )

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_path = 'checkpoint/age_dlfd.pth'
dlfm_model = load_model(8, device, model_path)

test_acc = calculate_accuracy(dlfm_model, data_loaders['test'], criterion, device)
mia = MIA(dlfm_model, data_loaders["retain_test"], data_loaders['forget_test'], data_loaders['test'], device)
final_score = (test_acc["Acc"] + 1 - abs(mia["Forgetting Score"] * 2)) / 2
print(f'Test Acc: {test_acc}')
print(f'MIA :{mia}')
print(f'Final score: {final_score}')

Test Acc: {'Loss': 0.03269348626441144, 'Acc': 0.6196808510638298, 'Top-2 Acc': 0.8736702127659575}
MIA :{'MIA Regression Accuracy': 0.7411744296990198, 'MIA CV Accuracy': 0.527906976744186, 'Forgetting Score': 0.027906976744185963}
Final score: 0.781933448787729
